PYTORCH로 딥러닝하기 60분만에 끝장내기

PyTorch 동적 그래프의 강력함을 보여주기 위해, 매우 이상한 모델을 구현해보겠습니다: 각 순전파 단계에서 4 ~ 5 사이의 임의의 숫자를 선택하여 다차항들에서 사용하고, 동일한 가중치를 여러번 재사용하여 4차항과 5차항을 계산하는 3-5차 다항식입니다.

In [30]:
import torch
import math
import random

class DynamicNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'



x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = DynamicNet()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)

for t in range(2000):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step() # 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다


print(f'Result: {model.string()}')

99 18331.888671875
199 745.0617065429688
299 1011.7971801757812
399 957.7861328125
499 76.4000473022461
599 36.567569732666016
699 23.707937240600586
799 21.38495445251465
899 20.366106033325195
999 19.172771453857422
1099 19.188919067382812
1199 19.511186599731445
1299 19.202699661254883
1399 18.52039337158203
1499 17.9853458404541
1599 18.053810119628906
1699 17.417356491088867
1799 16.521778106689453
1899 17.193540573120117
1999 16.959354400634766
Result: y = 0.09167798608541489 + 0.8780403137207031 x + -0.016946863383054733 x^2 + -0.09697654098272324 x^3 + 0.00015035620890557766 x^4 ? + 0.00015035620890557766 x^5 ?
